In [1]:
import cv2  # OpenCVで動画と画像処理
import torch  # PyTorchでYOLOv5モデル利用
import sys
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.manifold import MDS

In [2]:
# ========= 1. YOLOv5モデルのロード =========
#   yolov5s.ptは自動でダウンロードされます
#   deviceを'cpu'に明示することでCPU専用実行になります
yolo_model = torch.hub.load(
    './yolov5',          # クローンしたYOLOv5リポジトリのパス
    'custom',            # カスタムモデル（yolov5s等も可）
    path='yolov5s.pt',   # 学習済みモデル（デフォルトでOK）
    source='local',      # ローカルファイルとして読み込む
    device='cpu'         # CPU強制
)

# ラベル（人、車…などの名前リスト）
labels_list = list(yolo_model.names.values())

YOLOv5 🚀 v7.0-425-g85acef3a Python-3.12.3 torch-2.8.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [3]:
# ================= 1. Embedding =================
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')
embeddings = model.encode(labels_list, convert_to_numpy=True)

# ================= 2. 類似度 → 距離 =================
# cosine類似度
def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a)*np.linalg.norm(b))

n = len(labels_list)
sim_matrix = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        sim_matrix[i,j] = cosine_sim(embeddings[i], embeddings[j])

# 類似度 → 距離に変換（1-類似度）
dist_matrix = 1 - sim_matrix

# ================= 3. MDSで2D座標に埋め込み =================
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
coords = mds.fit_transform(dist_matrix)  # shape = (n_labels, 2)

# ================= 4. 座標 → 角度 =================
# 中心を原点に移動
coords_centered = coords - coords.mean(axis=0)
angles_rad = np.arctan2(coords_centered[:,1], coords_centered[:,0])
angles_deg = (np.degrees(angles_rad) + 360) % 360  # 0~360度に変換


/home/t-onuki/mov_recog/venv/lib/python3.12/site-packages/sklearn/manifold/_mds.py:677: FutureWarning: The default value of `n_init` will change from 4 to 1 in 1.9.
  warnings.warn(


In [7]:
class_names = []
base_angle_rad = angles_rad[0]
target_angle_rad = np.radians(210)
angles_rad_rotated = angles_rad + (target_angle_rad - base_angle_rad)
angles_deg_rotated = (np.degrees(angles_rad_rotated) + 360) % 360
for (k, v), angle in zip(yolo_model.names.items(), angles_deg_rotated):
    class_names.append({
        'label': v,
        'degree': float(angle)
    })
# class_names

- 可視化表現の設計

- 使えそうな入力変数
    - xc, yc -> 物体の中心
    - w -> 幅
    - h -> 高さ
    - score -> 信頼度
    - label -> 識別名
    - cls -> labelのインデックス


- 視覚変数
    - 位置：xc,yc
    - 色相: cls
    - 明度: 
    - 彩度: 
    - 透明度: 
    - 

opencvで使える画像メソッド

| 図形       | メソッド                                                                            | 主な引数                                                                          | 簡単な使い方                                                                                                     |
| -------- | ------------------------------------------------------------------------------- | ----------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------- |
| 線        | `cv2.line(img, pt1, pt2, color, thickness)`                                     | `pt1=(x1,y1)`, `pt2=(x2,y2)`, `color=(B,G,R)`, `thickness=線の太さ`               | `cv2.line(img, (50,50), (200,50), (0,255,0), 3)`                                                           |
| 矩形（四角形）  | `cv2.rectangle(img, pt1, pt2, color, thickness)`                                | `pt1=(左上x,左上y)`, `pt2=(右下x,右下y)`, `thickness=-1で塗りつぶし`                        | `cv2.rectangle(img, (50,50), (200,150), (255,0,0), 2)`                                                     |
| 円        | `cv2.circle(img, center, radius, color, thickness)`                             | `center=(x,y)`, `radius=半径`, `thickness=-1で塗りつぶし`                             | `cv2.circle(img, (150,150), 40, (0,0,255), -1)`                                                            |
| 楕円       | `cv2.ellipse(img, center, axes, angle, startAngle, endAngle, color, thickness)` | `center=(x,y)`, `axes=(長径/2, 短径/2)`, `angle=回転角度`, `startAngle/endAngle=描く範囲` | `cv2.ellipse(img, (200,200), (80,40), 30, 0, 360, (255,255,0), 2)`                                         |
| 多角形      | `cv2.polylines(img, [pts], isClosed, color, thickness)`                         | `pts = numpy.array([[x1,y1],[x2,y2],...], np.int32)`                          | `pts = np.array([[100,50],[200,150],[50,150]], np.int32); cv2.polylines(img, [pts], True, (0,255,255), 2)` |
| 塗りつぶし多角形 | `cv2.fillPoly(img, [pts], color)`                                               | `pts` は上と同じ                                                                   | `cv2.fillPoly(img, [pts], (0,128,255))`                                                                    |
| 文字       | `cv2.putText(img, text, org, font, fontScale, color, thickness)`                | `org=(x,y)`, `font=cv2.FONT_HERSHEY_SIMPLEX` など                               | `cv2.putText(img, "Hello", (50,250), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)`                       |


In [140]:
def hsv2bgr(h, s, v):
    # HSV色 (H=60°, S=255, V=255)
    hsv_color = np.uint8([[[h, s, v]]])  # OpenCVはH:0-179, S/V:0-255
    bgr_color = cv2.cvtColor(hsv_color, cv2.COLOR_HSV2BGR)[0][0]
    return tuple(int(c) for c in bgr_color)

def draw_info_on_mov(result_hist, frame, crnt_frame_num):
    alpha = 0.9
    height, width = frame.shape[:2]
    transparent_layer = np.zeros((height, width, 3), dtype=np.uint8)
    
    for result_dict in result_hist:
        results = result_dict['results']
        frame_num = result_dict['frame_num']
        for *box, conf, cls in results.xywh[0].tolist():
            # 座標とラベル
            cx, cy, w, h = map(int, box)
            label = class_names[int(cls)]['label']
            score = float(conf)

            damping_coef = frame_num / crnt_frame_num
            r = int((np.min([w, h]) * 0.1 + 10 * score ) * damping_coef)
            hue = class_names[int(cls)]['degree'] / 2
            saturation = 255 * 0.9 * damping_coef
            brightness = (255 - 100) * 0.9 * damping_coef + 100
            
            # 検出枠
            color = hsv2bgr(hue, saturation, brightness)
            cv2.circle(transparent_layer, (cx, cy), r, color, -1)
            cv2.circle(frame, (cx, cy), r, color, 2)

            if frame_num == crnt_frame_num:
                text = f'{"\n".join(label)}\n{score:.2f}'
                x0 = int(cx - r)
                y0 = int(cy - h/2)
                dy = 30
                for i, char in enumerate(reversed(text.split("\n"))):
                    y = y0 - i*dy
                    x = x0 + 20 if len(char) == 1 else x0
                    cv2.putText(frame, char, (x, y), cv2.FONT_HERSHEY_TRIPLEX, 1, (255,255,255), 1)
        
        display_frame = cv2.addWeighted(frame, 1.0, transparent_layer, alpha, 0)
    return display_frame

In [141]:
# ========= 2. 動画ファイルの読み込み =========
video_path = 'data/shinjuku.mp4' # 例：'input.mp4'  カメラなら 0
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("動画が開けません。ファイルパスを確認してください。")
    sys.exit(1)

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 1つ前のフレームの初期化
ret, frame = cap.read()
height, width = frame.shape[:2]
pre_shape_layer = np.zeros((height, width, 3), dtype=np.uint8)
pre_shape_layers = []

# 描画開始
result_hist = []
while True:
    ret, frame = cap.read()
    frame_num = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    crnt_frame_num = frame_num
    if not ret:
        break  # 動画の最後でループ終了

    # ========= 3. 推論（物体検出） =========
    # OpenCVはBGR⇒YOLOはRGB
    results = yolo_model(frame[..., ::-1])  # YOLOv5に画像を渡して検出
    result_hist.append({'results':results, 'frame_num': frame_num})
    
    # ========= 4. 結果をフレームに描画 =========
    # 動画はグレースケール化
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    
    # 視覚表現の追加
    display_frame = draw_info_on_mov(result_hist[-30:], gray_frame, crnt_frame_num)
    
    
    # ========= 5. ウィンドウ表示 =========
    cv2.imshow("YOLOv5 Detection", display_frame)
    # qキーで途中終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ========= 6. 後処理 =========
cap.release()
cv2.destroyAllWindows()

/home/t-onuki/mov_recog/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/t-onuki/mov_recog/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/t-onuki/mov_recog/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/t-onuki/mov_recog/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/t-onuki/mov_recog/yolov5/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` inste